In [2]:
from yahooquery import Ticker
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [11]:
class StockDetail():

    api_key = 'ADLK0ZK57SB9BJKX'

    def __init__(self, symbol):

        self.symbol = symbol
    # returns a bunch of information like a description of the company, marketcap, 52 week lows and highs, EPS, Dividend, etc.
    def get_company_overview(self):

        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        return response_json

    #retrieves current price of Stock
    def get_quote(self): 

        url = 'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if len(response_json) == 0:
            return "No data found."

        response_json = {x[4:].replace(' ','_'): v 
            for x, v in response_json['Global Quote'].items()}

        return response_json
    
    #monthly price data
    def get_monthly_adjusted(self):

        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol='+self.symbol+'&apikey='+self.api_key
        response = requests.get(url)
        response_json = response.json()

        if list(response_json.keys())[0] == 'Error Message':
            return "No data found."

        return response_json

    #live price data from yahooQuery
    def get_price(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.price
        df = pd.DataFrame.from_dict(raw_dict)

        return df
    
    #returns earnings trend data
    def get_earnings_trend(self):

        ticker = Ticker(self.symbol)
        raw_dict = ticker.earnings_trend

        output_dict = {}
        output_dict['currentQtr'] = raw_dict[self.symbol]['trend'][0]['epsTrend']
        output_dict['nextQtr'] = raw_dict[self.symbol]['trend'][1]['epsTrend']
        output_dict['currentYr'] = raw_dict[self.symbol]['trend'][2]['epsTrend']
        output_dict['nextYr'] = raw_dict[self.symbol]['trend'][3]['epsTrend']

        return output_dict

    def get_option_chain(self, **kwargs):

        ticker = Ticker(self.symbol)
        df = ticker.option_chain

        if type(df) is str:
            if df == 'No option chain data found':  
                return "No data found."
                
        df.reset_index(inplace=True)
        df = df[df['optionType']=='puts']

        expireDates = df['expiration'].dt.strftime("%Y-%m-%d").unique()    

        df['mid'] = ( ( df['bid'] + df['ask'] ) / 2 ).round(2)
        df['timeToExpire'] = (df['expiration'] - pd.Timestamp.today()).round('1d').dt.days + 1
        df['multiplier'] = 365 / df['timeToExpire']
        df['return'] = ( df['multiplier'] * df['mid'] * 100 ) / ( df['strike'] * 100 )
        df.drop(['optionType', 'contractSymbol', 'currency', 'contractSize', 'lastTradeDate', 'impliedVolatility', 'multiplier'], axis=1, inplace=True)
        df['change'] = df['change'].round(2)
        df['percentChange'] = df['percentChange'].round(2)
        df['return'] = (df['return'] * 100).round(2)

        # Set filters
        if kwargs['expiration'] != 'all' and kwargs['expiration'] != None:
            df = df[ df['expiration'] == kwargs['expiration'] ]

        if kwargs['strikeMin'] != '' and kwargs['strikeMin'] != None:
            df = df[ df['strike'] >= int(kwargs['strikeMin']) ]

        if kwargs['strikeMax'] != '' and kwargs['strikeMax'] != None:
            df = df[ df['strike'] <= int(kwargs['strikeMax']) ]

        if kwargs['returnMin'] != '' and kwargs['returnMin'] != None:
            df = df[ df['return'] >= int(kwargs['returnMin']) ]

        if kwargs['returnMax'] != '' and kwargs['returnMax'] != None:
            df = df[ df['return'] <= int(kwargs['returnMax']) ]

        if kwargs['timeToExpireMin'] != '' and kwargs['timeToExpireMin'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMin']) ]

        if kwargs['timeToExpireMax'] != '' and kwargs['timeToExpireMax'] != None:
            df = df[ df['timeToExpire'] >= int(kwargs['timeToExpireMax']) ]

        # Format Columns
        df['expiration'] = df['expiration'].dt.strftime("%Y-%m-%d")

        # Rename Columns
        column_rename = {
            'percentChange': '% Change',
            'openInterest': 'open Int'
        }
        df.rename(columns=column_rename, inplace=True)

        results = {}
        results['columns'] = df.columns.values
        results['expiration'] = expireDates
        results['data'] = df

        return results


In [3]:
#returns earnings trend data
def get_earnings_trend(symbol):

    ticker = Ticker(symbol)
    raw_dict = ticker.earnings_trend
    display(raw_dict)
    output_dict = {}
    output_dict['currentQtr'] = raw_dict[symbol]['trend'][0]['epsTrend']
    output_dict['nextQtr'] = raw_dict[symbol]['trend'][1]['epsTrend']
    output_dict['currentYr'] = raw_dict[symbol]['trend'][2]['epsTrend']
    output_dict['nextYr'] = raw_dict[symbol]['trend'][3]['epsTrend']

    return output_dict

get_earnings_trend('AAPL')

{'AAPL': {'trend': [{'maxAge': 1,
    'period': '0q',
    'endDate': '2023-03-31',
    'growth': -0.025999999,
    'earningsEstimate': {'avg': 1.48,
     'low': 1.32,
     'high': 1.69,
     'yearAgoEps': 1.52,
     'numberOfAnalysts': 26,
     'growth': -0.025999999},
    'revenueEstimate': {'avg': 97495400000,
     'low': 90057000000,
     'high': 106168000000,
     'numberOfAnalysts': 23,
     'yearAgoRevenue': 97278000000,
     'growth': 0.002},
    'epsTrend': {'current': 1.48,
     '7daysAgo': 1.49,
     '30daysAgo': 1.49,
     '60daysAgo': 1.49,
     '90daysAgo': 1.47},
    'epsRevisions': {'upLast7days': 2,
     'upLast30days': 5,
     'downLast30days': 4,
     'downLast90days': {}}},
   {'maxAge': 1,
    'period': '+1q',
    'endDate': '2023-06-30',
    'growth': 0.033,
    'earningsEstimate': {'avg': 1.24,
     'low': 1.0,
     'high': 1.43,
     'yearAgoEps': 1.2,
     'numberOfAnalysts': 26,
     'growth': 0.033},
    'revenueEstimate': {'avg': 87007500000,
     'low': 7883

{'currentQtr': {'current': 1.48,
  '7daysAgo': 1.49,
  '30daysAgo': 1.49,
  '60daysAgo': 1.49,
  '90daysAgo': 1.47},
 'nextQtr': {'current': 1.24,
  '7daysAgo': 1.25,
  '30daysAgo': 1.25,
  '60daysAgo': 1.26,
  '90daysAgo': 1.25},
 'currentYr': {'current': 6.14,
  '7daysAgo': 6.17,
  '30daysAgo': 6.19,
  '60daysAgo': 6.25,
  '90daysAgo': 6.26},
 'nextYr': {'current': 6.68,
  '7daysAgo': 6.73,
  '30daysAgo': 6.75,
  '60daysAgo': 6.81,
  '90daysAgo': 6.82}}

In [13]:
global AlphaVantageKey
#basic key AlphaVantageKey = 'XEQ9AFU8KM035KMG'
#premium api key
AlphaVantageKey = 'ZTB6U564ILR50HU3'


def fairValue_hist(ticker):
    global AlphaVantageKey
    global getFairValueTime    
    yq_ticker = Ticker(ticker)
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+AlphaVantageKey
    r = requests.get(url)
    data = r.json()
    # print(type(data['annualEarnings']))
    # display(data['fannualEarnings'])
    df = pd.DataFrame(data['annualEarnings'])
    df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
    filtered_values = np.where((df['fiscalDateEnding'] > '2017-01-01') & (df['fiscalDateEnding'] < '2020-01-01'))
    # display(filtered_values)
    eps1 = df.loc[filtered_values]
#     display(eps1)
    # print(filtered_values)

    testPrice = []
    reportedEPS_cols = []
    pe_cols = []
    testPrice_cols = []

    for index, row in eps1.iterrows():
        #retrieving testPrice from Past #change to 30-60 days average
        testPriceDateStart = row['fiscalDateEnding'] + relativedelta(months=+3)
        testPriceDateEnd = row['fiscalDateEnding'] + relativedelta(months=+4)
        df = yq_ticker.history(period='6y', interval='1d')
        priceHistory = pd.DataFrame(df)
        priceHistory = priceHistory.reset_index()
        priceHistory['date']= pd.to_datetime(priceHistory['date'])
        mask = (priceHistory['date'] > testPriceDateStart) & (priceHistory['date'] < testPriceDateEnd)
        avg_price = priceHistory.loc[mask]
        avgTestPrice = avg_price['close'].mean()
        testPrice.append(avgTestPrice)
        currentYear = row['fiscalDateEnding'].strftime("%Y")

        #making all the columns
        reportedEPS_cols.append('reportedEPS' + currentYear)
        pe_cols.append('p_e'+currentYear)
        testPrice_cols.append('avgPrice' + currentYear)
        testdf = pd.DataFrame(testPrice)
#         print('testdf')
#         display(testdf)

    #building vertical dataframe which ends up getting transposed
    eps1.reset_index(drop=True)
    eps1['testPrice'] = testdf.values
    eps1['reportedEPS'] = eps1['reportedEPS'].astype(float)
    eps1['p_e'] = eps1['testPrice']/eps1['reportedEPS']
    eps1['avgp_e'] = eps1['p_e'].mean()
    #current yr estimate
    currentYrEstimate = StockDetail(ticker).get_earnings_trend()['currentYr']['current']
    eps1['currentYrEstimate'] = currentYrEstimate
    eps1['FairValue'] = eps1['avgp_e'] * eps1['currentYrEstimate']
    
    #bulding long data lists which is what we actually use for fv_df
    data = eps1['reportedEPS'].to_list()
    data.extend(eps1['testPrice'].to_list())
    data.extend(eps1['p_e'].to_list())
    data.append(eps1['currentYrEstimate'].values[0])
    data.append(eps1['FairValue'].values[0])

    #creating final long data row from lists
    fv_df_cols = reportedEPS_cols + testPrice_cols + pe_cols + ['currentYrEstimate', 'FairValue']
#     print(fv_df_cols)
    fv_df = pd.DataFrame(data = [data], columns = fv_df_cols)

    # executionTime = (time.time() - startTime)
    # getFairValueTime = getFairValueTime + executionTime

    return fv_df

fairValue_hist('AAPL')

,reportedEPS2019,reportedEPS2018,reportedEPS2017,avgPrice2019,avgPrice2018,avgPrice2017,p_e2019,p_e2018,p_e2017,currentYrEstimate,FairValue
0,2.98,2.97,2.3,77.631375,38.29425,43.680395,26.050797,12.893687,18.991476,6.15,118.768718
